# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example processing workflow for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [ ]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from coastsat import NOC_tools, NOC_shoreline, NOC_download,\
    SDS_tools, SDS_shoreline, SDS_download, SDS_preprocess

settings ={'output_epsg': 32620,
            'reference_shoreline': True,
            'max_dist_ref': 100,        # max distance (in meters) allowed from the reference shoreline
          
          }

In [ ]:
# directory where the data will be accessed and stored
dataPartition = 'c:\\data\\coastsat'
country = 'STV'
base_filepath = os.path.join(dataPartition, country)  

# filepaths etc
filepath_sites = os.path.join(base_filepath, 'sites')
sites = os.listdir(filepath_sites)

sat_name = 'S1'

dates = ['2020-01-01', '2020-12-31']

for site in sites:

    kml_filepath = os.path.join(filepath_sites, site)
    kml_polygon = NOC_tools.polygon_from_kml(kml_filepath)
    roi_polygon = SDS_tools.smallest_rectangle(kml_polygon)
    
    sitename = site[:site.find('.')]

    inputs = {'polygon': roi_polygon, 'dates': dates, 'sat_list': [sat_name],
              'sitename': sitename, 'filepath': base_filepath, 'pixel_size': 10}
    
    settings['inputs'] = inputs
    
    print(f'Processing: {sitename}')

In [ ]:
    #metadata = NOC_download.retrieve_median_sar(inputs)
    ## load saved metadata
    metadata = NOC_download.get_metadata(settings['inputs'])
    
    print(metadata)
    print('finished ...')

In [ ]:
#SDS_preprocess.save_jpg(metadata, settings)

In [ ]:
%matplotlib qt
settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, settings)
settings['max_dist_ref'] = 100 # max distance (in meters) allowed from the reference shoreline

In [ ]:
    %matplotlib qt
    
    ##Batch shoreline detection
    output = NOC_shoreline.extract_sar_shorelines(metadata, settings)
    
    print('finished ...')

In [ ]:
output = SDS_tools.remove_duplicates(output) # removes duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_inaccurate_georef(output, 10) # remove inaccurate georeferencing (set threshold to 10 m)

In [ ]:
print(output.keys())

In [ ]:
gdf = SDS_tools.output_to_gdf(output, 'lines')

if  ~gdf.empty:
    gdf.crs = {'init':'epsg:'+str(settings['output_epsg'])} # set layer projection
    # save GEOJSON layer to file
    gdf.to_file(os.path.join(inputs['filepath'], inputs['sitename'],
                             '%s_output_%s.geojson'%(sitename,'lines')),
                             driver='GeoJSON', encoding='utf-8')
    print('finished ...')
else:
    print('no shorelines to be seen ...')

In [ ]:
fig = plt.figure(figsize=[15,8])
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
plt.legend();